In [ ]:
import os
import sys
from google.colab import drive
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Limpieza de la tabla de datos nutricionales por beneficiario**

####**Descripción:**

Filtra y limpia la tabla de datos nutricionales del beneficiario (primeras columnas de la tabla de datos nutricionales), integrando la información redundante cuando es posible e imputando datos faltantes cuando no lo es. Genera una nueva tabla de datos normalizada, con una fila por beneficiario.
 
####**Requerimientos:**

-  Diccionario `datatypes_dictionaries/dtypes_tomas.json`, que mapea cada columna de la tabla de datos nutricionales al *datatype* de Pandas más adecuado.
- Diccionario `datatypes_dictionaries/cat_tom.json`, que mapea cada columna categórica de la tabla de datos nutricionales a una lista (ordenada, si aplica) de sus categorías.
- Tabla de datos nutricionales preprocesada con los identificadores corregidos `clean_data/tomas_beneficiario_ids.parquet`.
- Tabla de datos sociodemográficos preprocesada con los identificadores corregidos `clean_data/sociodemo_ids.parquet`.
- Función `impute_nans_range_log` de la librería `utils`
 
####**Salidas:**

- Tabla de datos `clean_data/beneficiario.parquet` con datos del beneficiario completamente corregidos, y datos faltantes asignados.
- Tabla de datos de reporte `log_files/log_integracion_beneficiario.csv` con datos de los cambios hechos a la tabla dura.
- Archivo `log_files/log_inicio_beneficiario.txt` con el reporte de los datos nulos imputados al inicio.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
from collections import defaultdict
from utils import impute_nans_range_log

### **Lectura de archivos de datos**

In [ ]:
t0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_tomas.json', 'r') as file:
  dtypes_tomas = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/cat_tom.json', 'r') as file:
  cat_tom = json.load(file)
# Lee la tabla de datos nutricionales preprocesada con los identificadores correctos
tom = pd.read_parquet('clean_data/tomas_beneficiario_ids.parquet').\
  sort_values(['IdBeneficiario', 'Vigencia'])
for col in tom.columns:
  tom[col] = tom[col].astype(dtypes_tomas[col])
tom.info()
# Lee la tabla de datos sociodemográficos preprocesada con los identificadores correctos
cols_soc = ['FechaNacimiento', 'Sexo', 'ZonaUbicacion', 'PresentaDiscapacidad']
soc = pd.read_parquet('clean_data/sociodemo_ids.parquet')
# Lista de reportes de cambios realizados
log = list()
# Lista de tablas de reportes de cambios realizados (una tabla por columna)
log_data = list()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6007561 entries, 988318 to 1741421
Data columns (total 19 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Registro                        UInt32        
 1   Vigencia                        category      
 2   TipoBeneficiario                category      
 3   FechaNacimiento                 datetime64[ns]
 4   Sexo                            category      
 5   CodigoPaisNacimiento            category      
 6   PaisNacimiento                  category      
 7   PresentaDiscapacidad            category      
 8   ZonaUbicacion                   category      
 9   GrupoEtnico                     category      
 10  MesesLactanciaMaternaExclusiva  float32       
 11  MesesLactanciaMaternaTotal      float32       
 12  PesoAlNacer                     float32       
 13  TallaAlNacer                    float32       
 14  AntecedentePremadurez           category     

### **Preliminares y función de integración**

In [ ]:
# Duración promedio de un mes en días
dias_mes = 365.25 / 12

# Subconjuntos de variables útiles
# Lista de variables categóricas
cols_cat = [col for col in tom.columns if col in cat_tom.keys()]
# Columnas de medidas
cols_medidas = [
  'PesoAlNacer', 
  'TallaAlNacer',
  'MesesLactanciaMaternaExclusiva', 
  'MesesLactanciaMaternaTotal',
  'EdadGestacionalAlNacer'
  ]
# Columnas cuyo valor no varía con el tiempo
cols_constantes = [
  'IdBeneficiario',
  'Id',
  'FechaNacimiento',
  'Sexo',
  'PaisNacimiento',
  'PesoAlNacer',
  'TallaAlNacer',  
  'EdadGestacionalAlNacer',
  'GrupoEtnico',
  ]
# Columnas cuyo valor puede variar con el tiempo
cols_variables_cat = [
  'Tipo',
  'PresentaDiscapacidad',
  'ZonaUbicacion',
  'AntecedentePremadurez',
  'RegimenSeguridadSocial'
  ]
# Columnas cuyo valor relevante es el máximo presente
cols_variables_max = [
  'PresentaDiscapacidad', # Tomar 'S' > 'N'
  'MesesLactanciaMaternaExclusiva',
  'MesesLactanciaMaternaTotal'
  ]

# Si dos valores de la variable están así o más cercanos, se consideran indistinguibles
dif_min = {
    'TallaAlNacer': 5,  # 5 cm
    'PesoAlNacer': 0.1, # 100 g
    'FechaNacimiento': pd.to_timedelta(dias_mes, unit='day') # Un mes
    }
dif_min = defaultdict(lambda: np.nan, dif_min)
# Si dos valores de la variable están así o más lejanos, se consideran inconsistentes
dif_max = {
    'TallaAlNacer': 10, # 10 cm
    'PesoAlNacer': 1,   # 1 kg
    'FechaNacimiento': pd.to_timedelta(2 * dias_mes, unit='day')} # Dos meses
dif_max = defaultdict(lambda: np.nan, dif_max)
# Función de integración
def integrate(x, dif_min, dif_max):
  """
  Recibe un conjunto de datos 'x' que representa el valor de cierta variable para un beneficiario
  y retorna el valor más probable para esta variable, aplicando las siguientes reglas secuencialmente:
  - Si todos los datos son iguales, retorna este valor (método: None, detalle: None)
  - Si existe un dato mayoritario, retorna este valor (método: 'Moda', detalle: frecuencia de la moda)
  - Si la variable es numérica: 
    - Si el rango <= 'dif_min' (diferencia mínima), retorna la media
      (método: 'Diferencia mínima', detalle: rango)
    - Si 'dif_min' < rango <= 'dif_max' (diferencia máxima), retorna la media 
      (método: 'Diferencia considerable', detalle: rango)
    - Si rango > 'dif_max', retorna un valor nulo (NaN)
      (método: 'Diferencia inconsistente', detalle: rango)
  - Si la variable no es numérica, retorna un valor nulo (NaN) (método: 'Multimodal', detalle: rango)
  Retorna una 4-tupla: (valor final, método, número de registros, detalle)
  """
  # Si solo existe un valor (o ninguno), retorna este valor
  if x.nunique() <= 1:
    return  x.iloc[0], None, len(x), None
  # Si existe una moda definida, retorna la moda
  elif len(x.mode()) == 1:
    return x.mode()[0], 'Moda', len(x), len(x[x==x.mode()[0]])
  # Si no existe una moda definida y la variable es cualitativa, retorna un valor nulo
  elif pd.isna(dif_min) or pd.isna(dif_max):
    return np.nan, 'Multimodal', len(x), len(x.mode())
  # Si no existe una moda definida y la variable es cuantitativa...
  else:
    range = x.max() - x.min()
    # Si la variación entre los valores es mínima o aceptable, retorna la media
    if range <= dif_min:
      return x.mean(), 'Diferencia mínima', len(x), range
    elif range <= dif_max:
      return x.mean(), 'Diferencia significativa', len(x), range
    # Si la variación entre los valores es inaceptable, retorna un valor nulo
    else:
      return np.nan, 'Diferencia inconsistente', len(x), range  

### **Borrado de espacios y exploración de rangos**

In [ ]:
# Borra espacios al inicio y final de las categorías
for col in cols_cat[1:]:
  tom[col] = tom[col].astype('str').str.strip().astype('category')

In [ ]:
# Rangos de las columnas numéricas
tom.agg([min, max]).transpose()

min                  max
Registro                                          0              6007560
FechaNacimiento                 1900-01-01 00:00:00  2019-12-17 00:00:00
MesesLactanciaMaternaExclusiva                    0                   91
MesesLactanciaMaternaTotal                        0                   66
PesoAlNacer                                     0.5                   10
TallaAlNacer                                     32                   70
EdadGestacionalAlNacer                            0                   38
IdBeneficiario                                   36             18973750
Id                                                3              5945451

Existen fechas de nacimiento mal asignadas, con el año 1900 usado por defecto. Algo similar pasa con variables cuyo valor no puede ser físicamente iugal a cero, como el peso, la talla y la edad gestacional al nacer.

### **Imputación de valores nulos**

In [ ]:
# Imputa valores nulos (NaNs) para medidas de peso, talla y edad gestacional, cuyo valor no puede ser cero
for col in ['PesoAlNacer', 'TallaAlNacer', 'EdadGestacionalAlNacer']:
  nan_inicio = tom[col].isna().sum()
  tom[col] = tom[col].replace(0, np.NaN)
  nan_nuevos = tom[col].isna().sum() - nan_inicio
  print(f"En '{col}', {nan_nuevos} NaNs imputados: {time.time() - t0:.2f} s.")

En 'PesoAlNacer', 0 NaNs imputados: 37.78 s.
En 'TallaAlNacer', 0 NaNs imputados: 37.82 s.
En 'EdadGestacionalAlNacer', 3012159 NaNs imputados: 37.87 s.


In [ ]:
# Fecha de nacimiento mínima: Fecha cero de tiempo Unix (Unix epoch)
fecha_min = pd.to_datetime('1970 - 1 - 1')
# Fecha de nacimiento máxima: Beneficiario que nació al final del estudio
fecha_max = pd.to_datetime('2019 - 12 - 31')
rango = {
  'FechaNacimiento': [fecha_min, fecha_max],
  'PesoAlNacer': [0.5, 10],
  'TallaAlNacer': [20, 75],
  'MesesLactanciaMaternaExclusiva': [0, 18],
  'MesesLactanciaMaternaTotal': [0, 60],
  'EdadGestacionalAlNacer': [21, 44]
  }
# Reemplazar los valores por fuera de sus rangos naturales por NaNs
for col in rango.keys():
  nan_inicio = tom[col].isna().sum()
  tom[col] = impute_nans_range_log(data=tom, col=col, col_id='Registro', range=rango[col], log=log)
  nan_nuevos = tom[col].isna().sum() - nan_inicio
  print(f"'{col}': {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

'FechaNacimiento': 61 NaNs imputados: 63.79 s.
'PesoAlNacer': 0 NaNs imputados: 66.86 s.
'TallaAlNacer': 0 NaNs imputados: 70.00 s.
'MesesLactanciaMaternaExclusiva': 1 NaNs imputados: 73.47 s.
'MesesLactanciaMaternaTotal': 1 NaNs imputados: 76.90 s.
'EdadGestacionalAlNacer': 1 NaNs imputados: 80.02 s.


### **Validación de categorías**

In [ ]:
print('Categorías no presentes en el diccionario de datos:')
for col in tom.columns:
  if col in cols_cat:
    cat_nulas = tom[~(tom[col].isin(cat_tom[col]))][col].unique()
    print(f"'{col}': {list(cat_nulas)}")

Categorías no presentes en el diccionario de datos:
'Vigencia': []
'Sexo': ['-', 'I', 'nan']
'PresentaDiscapacidad': ['nan']
'ZonaUbicacion': ['nan']
'GrupoEtnico': ['nan']
'AntecedentePremadurez': ['nan', '']
'RegimenSeguridadSocial': ['nan']


Solo la variable `Sexo` presenta una categoría que no está en el diccionario de datos y que pudiera ser relevante (`I`: intersexual o sexo indeterminado). Se dejará como `NaN` para propósitos de este trabajo.

In [ ]:
# Forzar categorías
for col in cols_cat:
  categorias = cat_tom[col]
  nan_inicio = tom[col].isna().sum()
  tom[col] = tom[col].cat.set_categories(categorias, ordered=False)
  nan_nuevos = tom[col].isna().sum() - nan_inicio
  print(f"'{col}': {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

'Vigencia': 0 NaNs imputados: 81.22 s.
'Sexo': 1027 NaNs imputados: 81.25 s.
'PresentaDiscapacidad': 76237 NaNs imputados: 81.27 s.
'ZonaUbicacion': 79101 NaNs imputados: 81.30 s.
'GrupoEtnico': 161383 NaNs imputados: 81.33 s.
'AntecedentePremadurez': 4792241 NaNs imputados: 81.36 s.
'RegimenSeguridadSocial': 2173787 NaNs imputados: 81.39 s.


In [ ]:
tom.agg([min, max]).transpose()

min                  max
Registro                                          0              6007560
FechaNacimiento                 1973-12-23 00:00:00  2019-12-17 00:00:00
MesesLactanciaMaternaExclusiva                    0                   15
MesesLactanciaMaternaTotal                        0                   60
PesoAlNacer                                     0.5                   10
TallaAlNacer                                     32                   70
EdadGestacionalAlNacer                           22                   38
IdBeneficiario                                   36             18973750
Id                                                3              5945451

Todas las variables numéricas están ahora dentro de sus rangos naturales.

### **Integración de la información por beneficiario**

In [ ]:
# Tabla con una fila por beneficiario e identificadores
ben = tom[['IdBeneficiario', 'Id']].drop_duplicates()
ben = ben.set_index('IdBeneficiario')

# Para cada columna constante (a excepción de los identificadores) y para la zona de ubicación
for col in cols_constantes[2:] + ['ZonaUbicacion']:
  # Selecciona los datos de cada columna y los beneficiarios que tienen datos en esa columna
  data = tom[['IdBeneficiario', col]].dropna()
  IdB_set = set(data['IdBeneficiario'])
  # Tabla con los beneficiarios que tienen el mismo valor en todos sus registros (consistentes)
  # con los valores indexados por el 'IdBeneficiario'
  consistente = data.drop_duplicates().\
    drop_duplicates(subset=['IdBeneficiario'], keep=False).set_index('IdBeneficiario')
  IdBs_consistentes = set(consistente.index)
  # Beneficiarios que presentan valores distintos en sus registros (inconsistentes)
  IdBs_inconsistentes = {IdB for IdB in IdB_set if IdB not in IdBs_consistentes}
  # Aplica la función de integración a los beneficiarios con valores inconsistentes
  col_integrada = data[data['IdBeneficiario'].isin(IdBs_inconsistentes)].\
    groupby('IdBeneficiario')[col].apply(lambda x: integrate(x, dif_min[col], dif_max[col])).to_frame()
  # Asigna una columna a cada elemento de la 4-tupla: (valor final, método, número de registros, detalle)
  # con nombres 0, 1, 2, 3 y una columna con el nombre de la columna
  for i in range(4):
    col_integrada[i] = [row[i] for row in col_integrada[col]]
  col_integrada = col_integrada.drop(columns=[col])
  col_integrada['variable'] = col
  # Adiciona la tabla con los resultados de la función de integración a la lista de reporte
  log_data.append(col_integrada[['variable', 0, 1, 2, 3]].reset_index())
  del data
  # Concatena la serie con los valores finales a la tabla de beneficiarios
  # utilizando el 'IdBeneficiario' como índice
  nueva_col = pd.concat([consistente[col], 
                         col_integrada.rename(columns={0: col})[col]])
  ben[col] = nueva_col
  print(f"En '{col}', {col_integrada[1].notnull().sum()} cambios reportados: {time.time() - t0:.2f} s.")

En 'FechaNacimiento', 11281 cambios reportados: 111.82 s.
En 'Sexo', 6448 cambios reportados: 130.75 s.
En 'PaisNacimiento', 594 cambios reportados: 145.61 s.
En 'PesoAlNacer', 146 cambios reportados: 147.12 s.
En 'TallaAlNacer', 138 cambios reportados: 148.27 s.
En 'EdadGestacionalAlNacer', 207 cambios reportados: 148.59 s.
En 'GrupoEtnico', 172713 cambios reportados: 279.84 s.
En 'ZonaUbicacion', 97459 cambios reportados: 357.96 s.


In [ ]:
# Fracción de datos faltantes en la tabla de beneficiarios
ben.isna().mean()

Id                        0.551285
FechaNacimiento           0.000928
Sexo                      0.001288
PaisNacimiento            0.009915
PesoAlNacer               0.952240
TallaAlNacer              0.952559
EdadGestacionalAlNacer    0.995979
GrupoEtnico               0.036103
ZonaUbicacion             0.022572
dtype: float64

### **Asignación de variables especiales**

Existe una enorme cantidad de beneficiarios que, a lo largo del estudio, reportan alternativa pertenecer y no pertenecer a un grupo étnico minoritario (por ejemplo, datos de `AFROCOLOMBIANO` y `NO SE AUTORECONOCE EN NINGUNO DE LOS ANTERIORES` para un mismo beneficiario). En tales casos se asignará como grupo étnico aquel que sea minoritario (i.e. distinto a `NO SE AUTORECONOCE EN NINGUNO DE LOS ANTERIORES`).

In [ ]:
# Toma 'GrupoEtnico' como una variable ordinal, de tal manera que la categoría 
# 'NO SE AUTORECONOCE EN NINGUNO DE LOS ANTERIORES' se toma como matemáticamente 'mayor' a las demás.
cols_variables_min = ['GrupoEtnico']
tom['GrupoEtnico'] = tom['GrupoEtnico'].cat.as_ordered()
# Deja el valor mínimo de 'GrupoEtnico' descartando la categoría
# 'NO SE AUTORECONOCE EN NINGUNO DE LOS ANTERIORES' en caso de haber otra más
for col in cols_variables_min:
  nan_inicio = ben[col].isna().sum()
  data = tom[['IdBeneficiario', col]].dropna()
  col_integrada = data.groupby('IdBeneficiario')[col].min().to_frame()
  ben[col] = col_integrada[col]
  nan_resueltos = nan_inicio - ben[col].isna().sum()
  print(f"En '{col}', {nan_resueltos} datos resueltos: {time.time() - t0:.2f} s.")

En 'GrupoEtnico', 78238 datos resueltos: 750.88 s.


Siguiendo la misma lógica, se asume que un beneficiario presenta discapacidad si se reportó que la presentó al menos una vez durante el estudio.

Para asignar un tiempo de lactancia (exclusiva o total) a cada beneficiario, se toma el máximo valor reportado, para cada uno. Esto para permitir una interpretación en términos de qué tanto fue amamantado el beneficiario a lo largo de su vida, al menos durante el lapso estudiado, descartando los valores bajos, en los primeros meses de vida del beneficiario.

In [ ]:
# Toma 'PresentaDiscapacidad' como una variable ordinal, de tal manera que 'SI' > 'NO'.
tom['PresentaDiscapacidad'] = tom['PresentaDiscapacidad'].cat.as_ordered()
# Deja el valor máximo de las variables 'PresentaDiscapacidad' (si presentó discapacidad alguna vez),
# 'MesesLactanciaMaternaExclusiva' y 'MesesLactanciaMaternaTotal'
for col in cols_variables_max:
  data = tom[['IdBeneficiario', col]].dropna()
  col_integrada = data.groupby('IdBeneficiario')[col].max().to_frame()
  ben[col] = col_integrada[col]
  print(f"En '{col}', {col_integrada[col].notnull().sum()} valores asignados: {time.time() - t0:.2f} s.")

En 'PresentaDiscapacidad', 3329634 valores asignados: 1139.77 s.
En 'MesesLactanciaMaternaExclusiva', 2721355 valores asignados: 1141.94 s.
En 'MesesLactanciaMaternaTotal', 2721355 valores asignados: 1144.00 s.


Dada la naturaleza cambiante de estas variables, estas asignaciones no se guardan en ningún archivo.

In [ ]:
# Fracción de datos faltantes en la tabla de beneficiarios
ben.isna().mean()

Id                                0.551285
FechaNacimiento                   0.000928
Sexo                              0.001288
PaisNacimiento                    0.009915
PesoAlNacer                       0.952240
TallaAlNacer                      0.952559
EdadGestacionalAlNacer            0.995979
GrupoEtnico                       0.012835
ZonaUbicacion                     0.022572
PresentaDiscapacidad              0.009796
MesesLactanciaMaternaExclusiva    0.190693
MesesLactanciaMaternaTotal        0.190693
dtype: float64

### **Asignación de valores a partir la tabla de datos sociodemográficos**
Los datos faltantes de fecha de nacimiento, sexo, zona de ubicación y presencia de discapacidad intentan asignarse de acuerdo a lo encontrado en la tabla de datos sociodemográficos

In [ ]:
# Reemplaza algunas categorías en la tabla de datos sociodemográficos para homologarlas 
# a las presentes en la tabla de datos nutricionales
soc['Sexo'] = soc['Sexo'].replace({
    0: 'F', 
    1: 'M'
    }).astype('category')
soc['ZonaUbicacion'] = soc['Cod_clase'].replace({
    1: 'CABECERA',
    2: 'RESTO',
    3: 'RESTO'
    }).astype('category')
soc['PresentaDiscapacidad'] = soc['Ind_ninguna_discapac'].replace({
    0: 'SI',
    1: 'NO'
    }).astype('category')
# Para las columnas 'FechaNacimiento', 'Sexo', 'ZonaUbcacion' y 'PresentaDiscapacidad'
soc = soc[['IdBeneficiario'] + cols_soc].set_index('IdBeneficiario')
for col in cols_soc:
  # Para los registros con datos faltantes en la columna
  nan_inicio = ben[col].isna()
  # Asignarle los valores de la tabla de datos sociodemográficos
  nueva_col = ben.drop(columns=col).\
    merge(soc[[col]], right_index=True, left_index=True, how='left')[col]
  ben[col] = ben[col].fillna(nueva_col).astype(dtypes_tomas[col])
  nan_nuevos = ben[col].isna()
  # Guardar los datos asignados en una tabla de reporte y la adiciona a la lista de reporte con
  # método: 'Sociodemo', número de registros: 1, detalle: None.
  datos_nuevos = ben.loc[nan_inicio & ~nan_nuevos, col].to_frame()
  datos_nuevos = datos_nuevos.reset_index().rename(columns={col: 0})
  log_data.append(pd.DataFrame({
    'IdBeneficiario': datos_nuevos['IdBeneficiario'],
    'variable': [col] * len(datos_nuevos),
    1: 'Sociodemo',
    2: [1] * len(datos_nuevos),
    }))
  print(f"En '{col}', {len(datos_nuevos)} datos imputados: {time.time() - t0:.2f} s.")

En 'FechaNacimiento', 1012 datos imputados: 1149.22 s.
En 'Sexo', 1645 datos imputados: 1151.57 s.
En 'ZonaUbicacion', 32069 datos imputados: 1153.82 s.
En 'PresentaDiscapacidad', 8781 datos imputados: 1156.09 s.


### **Generación de la tabla de reporte**

In [ ]:
log_report = pd.concat(log_data)
log_report.columns = [
  'IdBeneficiario',
  'variable', 
  'valor_final',
  'metodo',
  'n_registros',
  'detalle'
  ]
log_report

IdBeneficiario              variable  ... n_registros             detalle
0             524191       FechaNacimiento  ...           3                   2
1             821675       FechaNacimiento  ...           2  1027 days 00:00:00
2             906752       FechaNacimiento  ...           2   509 days 00:00:00
3             983282       FechaNacimiento  ...           2  1102 days 00:00:00
4            1110976       FechaNacimiento  ...           2   188 days 00:00:00
...              ...                   ...  ...         ...                 ...
8776        18972973  PresentaDiscapacidad  ...           1                 NaN
8777        18972974  PresentaDiscapacidad  ...           1                 NaN
8778        18973007  PresentaDiscapacidad  ...           1                 NaN
8779        18973013  PresentaDiscapacidad  ...           1                 NaN
8780        18973021  PresentaDiscapacidad  ...           1                 NaN

[332493 rows x 6 columns]

In [ ]:
ben.info()
# Fracción de datos faltantes en la tabla de beneficiarios
ben.isna().mean()

<class 'pandas.core.frame.DataFrame'>
Index: 3362573 entries, 36 to 18973750
Data columns (total 12 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Id                              UInt32        
 1   FechaNacimiento                 datetime64[ns]
 2   Sexo                            category      
 3   PaisNacimiento                  object        
 4   PesoAlNacer                     float64       
 5   TallaAlNacer                    float64       
 6   EdadGestacionalAlNacer          float64       
 7   GrupoEtnico                     object        
 8   ZonaUbicacion                   category      
 9   PresentaDiscapacidad            category      
 10  MesesLactanciaMaternaExclusiva  float64       
 11  MesesLactanciaMaternaTotal      float64       
dtypes: UInt32(1), category(3), datetime64[ns](1), float64(5), object(2)
memory usage: 416.5+ MB


Id                                0.551285
FechaNacimiento                   0.000627
Sexo                              0.000798
PaisNacimiento                    0.009915
PesoAlNacer                       0.952240
TallaAlNacer                      0.952559
EdadGestacionalAlNacer            0.995979
GrupoEtnico                       0.012835
ZonaUbicacion                     0.013035
PresentaDiscapacidad              0.007184
MesesLactanciaMaternaExclusiva    0.190693
MesesLactanciaMaternaTotal        0.190693
dtype: float64

### **Guardado**

In [ ]:
# Guarda las tablas de datos de beneficiarios y de reporte de cambios
ben.reset_index().to_parquet('clean_data/beneficiario.parquet')
log_report.to_csv('log_files/log_integracion_beneficiario.csv')

# Escribe un archivo con los cambios generados
with open('log_files/log_inicio_beneficiario.txt', 'w') as log_file:
  log_file.write('Integracion_beneficiario:\n')
  for line in log:
    log_file.write('\t' + line + '\n')

print(f'Archivos guardados: {time.time() - t0:.2f} s. {len(log)} cambios guardados.')

Archivos guardados: 1162.25 s. 64 cambios guardados.
